# Model

In [20]:
import shutil
import urllib.request
import os
import cv2
import numpy as np
import yaml

In [21]:
if not os.path.exists('models'):
    os.makedirs('models')
if not os.path.exists('models/__init__.py'):
    open('models/__init__.py', 'a').close()

Each tracker must inherit from the `Tracker` class.

In [22]:
%%writefile model.py

import os

class Tracker:
    def __init__(self, model_name):
        self.model_name = model_name

    def initialize(self, image_file, box):
        pass

    def track(self, image_file):
        pass

Overwriting model.py


In [6]:
def clone_repo(model, url):
    path_zip = os.path.join('models', model + '.zip')
    path_no_zip = os.path.join('models')

    urllib.request.urlretrieve(url, path_zip)
    shutil.unpack_archive(path_zip, path_no_zip)
    os.rename(os.path.join(path_no_zip, model + '-master'), os.path.join(path_no_zip, model))
    os.remove(path_zip)

    open(os.path.join(path_no_zip, model, '__init__.py'), 'a').close()

## [SiamSE](https://github.com/isosnovik/SiamSE)

In [29]:
model = 'SiamSE'
url = 'https://github.com/ISosnovik/SiamSE/archive/refs/heads/master.zip'

clone_repo(model, url)

Install needed packages, present in the README.md of the SiamSE repo :


In [ ]:
!pip install yacs scipy shapely opencv-python numpy torchvision pyyaml shapely torch

Download the pretrained model [weights](https://drive.google.com/file/d/1WQ-9_QE9Xk9wj52vVcEDIXY2NBTupAnZ/view?usp=share_link) and put the file in the `SiamSE` folder.

In [23]:
%%writefile -a model.py

import os
import cv2

def get_SESIAMFC_tracker():
    import yaml
    import numpy as np
    import models.SiamSE.lib.models.models as models
    from models.SiamSE.lib.tracker import SESiamFCTracker
    from models.SiamSE.lib.utils import load_pretrain, cxy_wh_2_rect, convert_color_RGB
    
    def get_axis_aligned_bbox(bbox):
        """Convert bbox to [xc, yc, w, h] format"""
        x, y, w, h = bbox
        return [x + w / 2, y + h / 2, w, h]
    
    class SEsiamfc(Tracker):
        def __init__(self):
            super().__init__("SEsiamfc")

            path_SiamSE = 'models/SiamSE'
            model_pretrained_path = os.path.join(path_SiamSE, 'checkpoint_vot.pth')
            config_path = os.path.join(path_SiamSE, 'configs/test.yaml')
            
            model_carac = 'VOT2017'

            with open(config_path, 'r') as f:
                tracker_config = yaml.load(f.read(), Loader=yaml.FullLoader)

            # prepare model
            net = models.__dict__[tracker_config['MODEL']](padding_mode='constant')
            net = load_pretrain(net, model_pretrained_path)
            net = net.eval().cuda()

            # prepare tracker
            tracker_config = tracker_config['TRACKER'][model_carac]
            self.tracker = SESiamFCTracker(net, **tracker_config)

        def initialize(self, image_file, box):
            # prepare image
            image = cv2.imread(image_file)
            image = convert_color_RGB(image)
            
            # prepare box
            cx, cy, w, h = get_axis_aligned_bbox(box)
            target_pos = np.array([cx, cy])
            target_sz = np.array([w, h])

            self.tracker.init(image, target_pos, target_sz)

        def track(self, image_file):
            # prepare image
            image = cv2.imread(image_file)
            image = convert_color_RGB(image)
            
            # track
            target_pos, target_sz = self.tracker.track(image)

            # prepare bbox
            bbox = cxy_wh_2_rect(target_pos, target_sz)
            return bbox
    
    return SEsiamfc()

Appending to model.py


## [AAA](https://github.com/songheony/A3T)

In [32]:
model = 'A3T'
url = 'https://github.com/songheony/A3T/archive/refs/heads/master.zip'

clone_repo(model, url)

In [ ]:
%pip install opencv-python opencv-contrib-python Cython seaborn sympy

%cd models/A3T

# clone frameworks
!git clone https://github.com/songheony/pytracking.git
!git clone https://github.com/StrangerZhang/pysot-toolkit

# install region
%cd pysot-toolkit/pysot/utils/
!python setup.py build_ext --inplace
%cd ../../..

%cd ../../

Some modification on the path used has to be performed.

In [24]:
%%writefile -a model.py

def get_AAA_tracker():
    import numpy as np
    class AAA(Tracker):
        def __init__(self):
            super().__init__("AAA")
            import models.A3T.algorithms.aaa as aaa
            self.n_experts = 6
            theta, gamma = 0.92, 11  # you can tune hyperparameters by running run_tuning.sh
            self.tracker = aaa.AAA(self.n_experts, mode="LOG_DIR", threshold=theta)

        def initialize(self, image_path, bbox):
            self.tracker.initialize(image_path, bbox)

        def track(self, image_path):
            experts_result = np.zeros((self.n_experts, 4))
            state, offline, weight = self.tracker.track(image_path, experts_result)
            return state
  
    return AAA()

Appending to model.py


# All

In [25]:
%%writefile -a model.py

def load_models():
    ret = {}
    
    ret['SEsiamFC'] = get_SESIAMFC_tracker()
    ret['AAA'] = get_AAA_tracker()
    
    return ret

Appending to model.py


# 